# Simple Linear Regression

<hr>

Sergei Yu. Papulin (papulin.study@yandex.ru)

<a name="0"></a>
<div><span style="font-size:16pt; font-weight:bold">Contents</span>
    <ol>
        <li><a href="#1">Loading Initial Data</a></li>
        <li><a href="#2">Defining Linear Regression Task</a></li>
        <li><a href="#3">Ordinary Least Squares</a></li>
        <li><a href="#4">Gradient Descent</a></li>
        <li><a href="#5">Stochastic Gradient Descent</a></li>
        <li><a href="#6">References</a></li>
    </ol>
</div>

Import modules and functions that will be used later

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
%matplotlib inline

<a name="1"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:16pt; font-weight:bold">1. Loading Initial Data</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">To contents</a></div>
    </div>
</div>

Create a dataframe from the csv-file of student grades and show the first 5 records/samples:

In [ ]:
FILE_PATH = "../data/SAT_GPA.csv"

In [ ]:
df1 = pd.read_csv(FILE_PATH, sep=" ")
df1.head(5)

Display a `high_GPA`-`univ_GRA` scatter plot for the loaded data:

In [ ]:
# Scatter plot

sx = df1["high_GPA"]
sy = df1["univ_GPA"]

plt.figure("1", figsize=[10, 6])

plt.subplot(1,1,1)

plt.scatter(sx, sy, color="slategrey")
plt.xlabel("High_GPA")
plt.ylabel("Univ_GPA")
plt.axis([2, 4, 2, 4])
plt.grid(True)


# Single Point

xA = df1.loc[36, "high_GPA"]
yA = df1.loc[36, "univ_GPA"]

xyA = "$("+str(xA)+","+str(yA)+")$"
plt.annotate(xyA, xy=(xA, yA), xytext=(50, -100), xycoords="data", textcoords="offset points", 
             arrowprops=dict(arrowstyle="->", connectionstyle="arc,angleA=0,armA=0,angleB=-90,armB=15,rad=7"),)
plt.show()

<a name="2"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:16pt; font-weight:bold">2. Defining Linear Regression Task</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">To contents</a></div>
    </div>
</div>

In [ ]:
import ipywidgets as widgets

In [ ]:
x = np.linspace(2, 4, num=100)

w0 = widgets.FloatSlider(min=-1.5, max=1.2, step=0.05, value=0.75)
w1 = widgets.FloatSlider(min=0.4, max=2.0, step=0.05, value=0.75)

def update(w0=0, w1=0):
   
    plt.figure("2", figsize=[10, 6])  
    plt.scatter(sx, sy, color="slategrey", label="Samples")
    plt.xlabel("High_GPA")
    plt.ylabel("Univ_GPA")
    plt.plot(x, w1 * x + w0, color="darkorange", linewidth=2, label="Regression Line")
    plt.axis([2, 4, 2, 4])
    plt.grid(True)
    plt.ylim(2, 4)
    plt.legend()
    plt.show()

widgets.interact(update, w0=w0, w1=w1);

Let's consider three linear functions and determine which one is the best of then for prediction:

1. $h_1\left( x \right) = -3 + 2x$

2. $h_2\left( x \right) = 1.1 + 0.7x$

3. $h_3\left( x \right) = 2 + 0.4x$

Initialize regression parameters of the functions:

In [ ]:
slope_1 = 2.0; slope_2 = 0.7; slope_3 = 0.4
intercept_1 = -3.0; intercept_2 = 1.1; intercept_3 = 2.0

Create python functions to compute predictions:   

In [ ]:
h_pred_1 = lambda x: slope_1 * x + intercept_1
h_pred_2 = lambda x: slope_2 * x + intercept_2
h_pred_3 = lambda x: slope_3 * x + intercept_3

Show the functions on the plot:

In [ ]:
plt.figure("2", figsize=[10, 6])

ax = plt.subplot(1,1,1)


# Samples

plt.xlabel("High_GPA")
plt.ylabel("Univ_GPA")
plt.scatter(sx, sy, color="slategrey", label="Samples")


# Linear Functions

x_line = np.array([1.5, 4.5])
plt.plot(x_line, h_pred_1(x_line), "-", label="$h_1(x)=0.9\cdot x+0.3$")
plt.plot(x_line, h_pred_2(x_line), "-", label="$h_2(x)=0.7\cdot x+1.1$")
plt.plot(x_line, h_pred_3(x_line), "-", label="$h_3(x)=0.4\cdot x+2.0$")


plt.axis([2, 4, 2, 4])
plt.grid(True)
ax.set_axisbelow(True)

plt.legend()

plt.show()

Which of the functions gives the best prediction?

In [ ]:
y_pred_1 = h_pred_1(sx)
y_pred_2 = h_pred_2(sx)
y_pred_3 = h_pred_3(sx)

In [ ]:
err_1 = ((sy - y_pred_1)**2).sum()
err_2 = ((sy - y_pred_2)**2).sum()
err_3 = ((sy - y_pred_3)**2).sum()

err_1, err_2, err_3

$$\hat \theta_0, \hat \theta_1 =\operatorname*{arg\,min}_{\theta_0, \theta_1} 
\displaystyle\sum_{i=1}^{N} ( y_i - (\theta_0+\theta_1 x_i))^2$$

<a name="3"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:16pt; font-weight:bold">3. Ordinary Least Squares</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">To contents</a></div>
    </div>
</div>

Estimates of parameters:

$$\hat{\theta}=\left(\begin{matrix}{\hat{\theta}}_0\\\begin{matrix}{\hat{\theta}}_1\\\vdots\\\end{matrix}\\{\hat{\theta}}_p\\\end{matrix}\right)=\left(X^TX\right)^{-1}X^Ty$$

Create $X$ and $\mathrm{y}$ matrices:

In [ ]:
X = np.c_[np.ones(df1["high_GPA"].size), df1["high_GPA"].values]
X[:5,:2]

In [ ]:
y = df1["univ_GPA"].values.reshape(df1["univ_GPA"].values.size, 1)
y[:5]

Now apply the formula for calculating regression parameters $\theta$:

In [ ]:
# %%timeit -r 7
w = np.linalg.inv(X.T @ X) @ X.T @ y
w

Alternatively, the `linalg` module has the built-in function for finding the least-squares solution. Use it to get parameters:

In [ ]:
# %%timeit -r 7
w, residuals, rank, s = np.linalg.lstsq(X, y, rcond=None)
w

In [ ]:
w0, w1 = w[0,0], w[1,0]
w0, w1

Define the prediction function:

In [ ]:
h_pred_ols = lambda x: w1 * x + w0

Plot the graph of the regression line:

In [ ]:
plt.figure("3", figsize=[10, 6])

ax = plt.subplot(1,1,1)


# Samples

plt.scatter(sx, sy, color="slategrey", label="Samples")
plt.xlabel("High_GPA")
plt.ylabel("Univ_GPA")


# Regression Lines

x_line = np.array([1.5, 4.5])
plt.plot(x_line, h_pred_1(x_line), "-", label="$f_1(x)=0.9\cdot x+0.3$")
plt.plot(x_line, h_pred_2(x_line), "-", label="$f_2(x)=0.7\cdot x+1.1$")
plt.plot(x_line, h_pred_3(x_line), "-", label="$f_3(x)=0.4\cdot x+2.0$")
plt.plot(x_line, h_pred_ols(x_line), "-", linewidth=4, 
         label="$f_{OLS}(x)$")


plt.axis([2, 4, 2, 4])
plt.grid(True)
plt.legend()

plt.show()

In [ ]:
err_ols = ((h_pred_ols(sx) - sy)**2).sum()
err_ols

In [ ]:
err_1, err_2, err_3, err_ols

<a name="4"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:16pt; font-weight:bold">4. Gradient Descent</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">To contents</a></div>
    </div>
</div>

Define the loss/cost function and its partial derivatives:

In [ ]:
# The number of sample elements
n = len(sx)

# Cost function 
mse = lambda x, y, w0, w1: 1 / n * sum([(y[i] - w1 * x[i] - w0) ** 2 for i in range(n)])

# Partial derivative of err over w0 and w1
del_w0 = lambda x, y, w0, w1: 2 / n * sum([-1 * (y[i] - w1 * x[i] - w0) for i in range(n)])
del_w1 = lambda x, y, w0, w1: 2 / n * sum([-x[i] * ( y[i] - w1 * x[i] - w0) for i in range(n)])

Write a function implimneting the gradient descent method:

In [ ]:
# X_ = np.c_[X[:,:1], (X[:,1:] - X[:,1:].mean(axis=0)) / X[:,1:].std(axis=0)]
# X_[:5]

In [ ]:
def GD(x, y, max_iter=200, min_err=0.0001, alpha=0.05, err=None):
    """
    Naive Gradient Descent (single feature)
    
    Parameters
    ----------
    alpha: step size
    max_iter: the maximum numbers of iterations
    min_err: a minimal change of cost error
    err: a cost function
    
    Returns
    -------
    
    
    """
    i = 0  # current iteration
    w0 = 0; w1 = 0  #  initial parameters
    w0_prev = 4; w1_prev = 4  # start point

    while i < max_iter:

        w0 = w0_prev - alpha * del_w0(x, y, w0_prev, w1_prev)
        w1 = w1_prev - alpha * del_w1(x, y, w0_prev, w1_prev)

        if abs(err(x, y, w0, w1) - err(x, y, w0_prev, w1_prev)) <= min_err:
            break

        w0_prev = w0
        w1_prev = w1

        i += 1

    return (w0, w1, i)

In [ ]:
def compute_gradient_descent(X, y, 
                             max_iter=200, 
                             min_err=0.0001, 
                             alpha=0.05, 
                             initial_w=None):
    """
    Naive Gradient Descent (multiple features)
    """
    
    # initialize variable
    w = initial_w.reshape(-1, 1) if initial_w is not None else np.zeros((X.shape[1],1))
    grad = np.zeros((X.shape[1],1))
    n = y.shape[0]
    loss = 1.0/n * np.sum((X @ w - y)**2)
    for i in range(max_iter):
        # compute gradient
        grad = X.T @ (X @ w - y)
        # update weights
        w = w - alpha * 2.0/n * grad
        # calculate loss
        loss_ = 1.0/n * np.sum((X @ w - y)**2)
        # check stop criteria
        if abs(loss - loss_) <= min_err:
            break
        loss = loss_
    return (*w.reshape(-1).tolist(), i+1)

Launch parameters estimation using GD

In [ ]:
w0_gd, w1_gd, i = GD(sx, sy, err=mse)
w0_gd, w1_gd, i

In [ ]:
compute_gradient_descent(X, y, initial_w=np.array([4., 4.]))

Specify the prediction function with the found parameters:

In [ ]:
h_pred_gd = lambda x: w1_gd * x + w0_gd

Plot the prediction line:

In [ ]:
x_line = np.array([1.5, 4.5])

plt.figure("3", figsize=[10, 6])

ax = plt.subplot(1,1,1)

plt.scatter(sx, sy, color="slategrey", label="Samples")
plt.plot(x_line, h_pred_2(x_line), "-", label="$f_2(x)$")
plt.plot(x_line, h_pred_ols(x_line), "-", linewidth=4, 
         label="$f_{OLS}(x)$")
plt.plot(x_line, h_pred_gd(x_line), "-", linewidth=2, color="black",
         label="$f_{GD}(x)$")

plt.xlabel("High_GPA")
plt.ylabel("Univ_GPA")

plt.axis([2, 4, 2, 4])

plt.grid(True)
ax.set_axisbelow(True) 

plt.legend()

plt.show()

Calculate the MSE for the prediction function:

In [ ]:
err_gd = ((sy - h_pred_gd(sx))**2).sum()
err_gd

Compare errors:

In [ ]:
err_1, err_2, err_3, err_ols, err_gd

What's wrong? Plot the contour graphic of the cost function:

In [ ]:
coord_w0 = np.arange(-4, 5, 0.1)
coord_w1 = np.arange(-4, 5, 0.1)

W0, W1 = np.meshgrid(coord_w0, coord_w1)

In [ ]:
W0

In [ ]:
coord_w0_large = np.arange(-400, 500, 1)
coord_w1_large = np.arange(-400, 500, 1)

W0_large, W1_large = np.meshgrid(coord_w0_large, coord_w1_large)

In [ ]:
plt.figure("12",figsize=[12, 4])

plt.subplot(1,2,1)
plt.contour(W0, W1, mse(sx, sy, W0, W1), 20, cmap=cm.bwr, alpha=0.5)
plt.grid(True)
plt.colorbar()
plt.xlabel("$\\theta_0$")
plt.ylabel("$\\theta_1$")

plt.subplot(1,2,2)
plt.contour(W0_large, W1_large, mse(sx, sy, W0_large, W1_large), 20, cmap=cm.bwr, alpha=0.5)
plt.grid(True)
plt.colorbar()
plt.xlabel("$\\theta_0$")
plt.ylabel("$\\theta_1$")

plt.show()

### Feature standardization

$$ x_s = \frac{x - \bar{x}}{s} $$

In [ ]:
sx_stand = (sx - sx.mean()) / sx.std()
sx_stand.head(5)

In [ ]:
x_line = np.array([1.5, 4.5])

plt.figure("3", figsize=[12, 4])

ax = plt.subplot(1,2,1)

plt.scatter(sx, sy, color="slategrey", label="Samples")
plt.title("Initial")
plt.xlabel("High_GPA")
plt.ylabel("Univ_GPA")

plt.axis([2, 4, 2, 4])

plt.grid(True)
ax.set_axisbelow(True) 

plt.legend()

ax = plt.subplot(1,2,2)

plt.scatter(sx_stand, sy, color="slategrey", label="Samples")
plt.title("Standardized")
plt.xlabel("High_GPA")
plt.ylabel("Univ_GPA")

plt.grid(True)
ax.set_axisbelow(True) 

plt.legend()


plt.show()

In [ ]:
plt.figure("12", figsize=[12, 4])

plt.subplot(1,2,1)
plt.contour(W0, W1, mse(sx, sy, W0, W1), 20, cmap=cm.bwr, alpha=0.5)
plt.title("Initial")
plt.xlabel("$\\theta_0$")
plt.ylabel("$\\theta_1$")
plt.grid(True)
plt.colorbar()

plt.subplot(1,2,2)
plt.contour(W0, W1, mse(sx_stand, sy, W0, W1), 20, cmap=cm.bwr, alpha=0.5)
plt.title("Standardized")
plt.xlabel("$\\theta_0$")
plt.ylabel("$\\theta_1$")
plt.grid(True)
plt.colorbar()

plt.show()

Repeat training with the GD for the standardized feature:

In [ ]:
w0_gd_stand, w1_gd_stand, i = GD(sx_stand, sy, err=mse)
w0_gd_stand, w1_gd_stand, i

Specify the prediction function:

In [ ]:
h_pred_gr_stand = lambda x: w1_gd_stand * x + w0_gd_stand

Display the scatter plot:

In [ ]:
x_line_stand = np.array([-2, 2])

plt.figure("3", figsize=[10, 6])

ax = plt.subplot(1,1,1)

plt.scatter(sx_stand, sy, color="slategrey", label="Samples")
plt.plot(x_line_stand, h_pred_gr_stand(x_line_stand), "-", linewidth=2, color="black",
         label="$f_{StndGD}(x)$")

plt.xlabel("High_GPA")
plt.ylabel("Univ_GPA")

plt.grid(True)
ax.set_axisbelow(True) 

plt.legend()

plt.show()

Recover the initial scale:

In [ ]:
w1_gd_rec = w1_gd_stand / sx_std
w1_gd_rec

In [ ]:
w0_gd_rec = w0_gd_stand - w1_gd_stand * sx_mean / sx_std
w0_gd_rec

Specify the prediction function:

In [ ]:
h_pred_gr_stand_recover = lambda x: w0_gd_rec + w1_gd_rec * x

Plot the prediction line:

In [ ]:
x_line = np.array([1.5, 4.5])

plt.figure("3", figsize=[10, 6])

ax = plt.subplot(1,1,1)

plt.scatter(sx, sy, color="slategrey", label="Samples")
plt.plot(x_line, h_pred_2(x_line), "-", label="$f_2(x)$")
plt.plot(x_line, h_pred_ols(x_line), "-", linewidth=4, 
         label="$f_{OLS}(x)$")
plt.plot(x_line, h_pred_gd(x_line), "-", linewidth=2, color="black",
         label="$f_{GD}(x)$")
plt.plot(x_line, h_pred_gr_stand_recover(x_line), "-", linewidth=2, color="cyan",
         label="$f_{StdGD}(x)$")

plt.xlabel("High_GPA")
plt.ylabel("Univ_GPA")

plt.axis([2, 4, 2, 4])

plt.grid(True)
ax.set_axisbelow(True) 

plt.legend()

plt.show()

Calculate the RSS:

In [ ]:
err_gd_stand = ((sy - h_pred_gr_stand_recover(sx))**2).sum()
err_gd_stand

In [ ]:
err_1, err_2, err_3, err_ols, err_gd, err_gd_stand

<a name="5"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:16pt; font-weight:bold">5. Stochastic Gradient Descent</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">To contents</a></div>
    </div>
</div>

Define the loss/cost function and its partial derivatives:

In [ ]:
# The number of sample elements
n = len(sx)

# Cost function 
mse = lambda x, y, w0, w1: 1 / n * sum([(y[i] - w1*x[i] - w0) ** 2 for i in range(n)])

# Partial derivative of err over w0 and w1
del_w0_i = lambda x, y, w0, w1, i: -2.0 * (y[i] - w0 - w1*x[i])
del_w1_i = lambda x, y, w0, w1, i: -2.0 * x[i] * (y[i] - w0 - w1*x[i])

In [ ]:
def SGD(x, y, min_err=0.000001, alpha=0.05, err=None):
    """
    Naive Stochastic Gradient Descent
    
    Parameters
    ----------
    alpha: step size
    min_err: a minimal change of cost error
    
    Returns
    -------
    w0: intercept
    w1: slope
    i: the number of iteration
    
    """

    i = 0  # current iteration
    w0 = 0; w1 = 0  #  initial parameters
    w0_prev = 4; w1_prev = 4  # start point
    
    err_prev = err(x, y, w0, w1)
    err_cur = err_prev

    for j in range(n):
        
        w0 = w0_prev - alpha * del_w0_i(x, y, w0_prev, w1_prev, j)
        w1 = w1_prev - alpha * del_w1_i(x, y, w0_prev, w1_prev, j)

        err_cur = err(x, y, w0, w1)

        if abs(err_cur - err_prev) <= min_err:
                break

        err_prev = err_cur    

        w0_prev = w0
        w1_prev = w1

        i += 1

    return (w0, w1, j)

Estimate regression parameters using the SGD:

In [ ]:
w0_sgd_stand, w1_sgd_stand, i = SGD(sx_stand, sy, err=mse)
w0_sgd_stand, w1_sgd_stand, i

Specify the prediction function with the found parameters:

In [ ]:
h_pred_sgd = lambda x: w1_sgd_stand * x + w0_sgd_stand

Plot the prediction line:

In [ ]:
x_line_stand = np.array([-2, 2])

plt.figure("3", figsize=[10, 6])

ax = plt.subplot(1,1,1)

plt.scatter(sx_stand, sy, color="slategrey", label="Samples")
plt.plot(x_line_stand, h_pred_sgd(x_line_stand), "-", linewidth=2, color="black",
         label="$f_{StndGD}(x)$")

plt.xlabel("High_GPA")
plt.ylabel("Univ_GPA")

plt.grid(True)
ax.set_axisbelow(True) 

plt.legend()

plt.show()

Recover the initial scale:

In [ ]:
w1_sgd_rec = w1_sgd_stand / sx_std
w1_sgd_rec

In [ ]:
w0_sgd_rec = w0_sgd_stand - w1_sgd_stand * sx_mean / sx_std
w0_sgd_rec

Specify the prediction function:

In [ ]:
h_pred_sgd_stand_recover = lambda x: w0_sgd_rec + w1_sgd_rec * x

Plot the prediction line:

In [ ]:
x_line = np.array([1.5, 4.5])

plt.figure("3", figsize=[10, 6])

ax = plt.subplot(1,1,1)

plt.scatter(sx, sy, color="slategrey", label="Samples")
plt.plot(x_line, h_pred_ols(x_line), "-", linewidth=4, 
         label="$h_{OLS}(x)$")
plt.plot(x_line, h_pred_gr_stand_recover(x_line), "-", linewidth=1, color="cyan",
         label="$h_{StdGD}(x)$")
plt.plot(x_line, h_pred_sgd_stand_recover(x_line), "-", linewidth=4, color="darkmagenta",
         label="$h_{StdSGD}(x)$")

plt.xlabel("High_GPA")
plt.ylabel("Univ_GPA")

plt.axis([2, 4, 2, 4])

plt.grid(True)
ax.set_axisbelow(True) 

plt.legend()

plt.show()

Calculate the MSE:

In [ ]:
err_sgd_stand = ((sy - h_pred_sgd_stand_recover(sx))**2).sum()
err_sgd_stand

In [ ]:
err_1, err_2, err_3, err_ols, err_gd, err_gd_stand, err_sgd_stand

## Brute-Force Search

In [ ]:
mse(sx, sy, W0, W1)

In [ ]:
mse(sx, sy, W0, W1).min()

In [ ]:
inxd_ws = np.argwhere(mse(sx, sy, W0, W1).min() == mse(sx, sy, W0, W1))
inxd_ws

In [ ]:
W0[inxd_ws[0,0], inxd_ws[0,1]], W1[inxd_ws[0,0], inxd_ws[0,1]]

<a name="6"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:16pt; font-weight:bold">6. References</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">To contents</a></div>
    </div>
</div>